# super questionable code for reviewing data. make sure to leave ship name in!

In [98]:
start_ind = -50
end_ind = 0

In [100]:
start_ind += 50
end_ind += 50
cliwoc_data[start_ind:end_ind]

,VoyageFrom,VoyageTo,ShipName,ShipType,Nationality,Year,slave_logs
LogbookIdent,,,,,,,
50,Helvoetsluis,Curacao,kenau hasselaar,NaN,Dutch,1802,0
51,Porte Praijo,Schapeneiland,havick,Sloep,Dutch,1796,0
52,Batavia,Europa,havick,Korvet,Dutch,1810,0
53,Frankrijk,Nederlands Indië,nymphe,Fregat,French,1811,0
54,Batavia,Nieuwediep,jan en cornelis,Koopvaarder,Dutch,1821,0
55,Suriname,Nieuwediep,boreas,Fregat,Dutch,1773,2
56,St Eustacius,Texel,jason,NaN,Dutch,1779,1
57,Texel,Fonciaal,nassau-weilburgh,NaN,Dutch,1780,0
58,Lissabon,Mesurado,thetis,Fregat,Dutch,1787,0


# TO DO:
- Mess with classifier parameters (once working)
- test classifier for 2 validation sets


In [1]:
import collections

import numpy as np
import pandas as pd
import exploringShipLogbooks.wordCount as wc

from fuzzywuzzy import fuzz
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB

from exploringShipLogbooks.basic_utils import clean_data
from exploringShipLogbooks.basic_utils import encode_data_df
from exploringShipLogbooks.basic_utils import extract_logbook_data
from exploringShipLogbooks.basic_utils import isolate_columns
from exploringShipLogbooks.basic_utils import isolate_training_data

from exploringShipLogbooks.config import *

C:\Users\Emma\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Load and clean data
### Load CLIWOC ship logs

In [2]:
# extract data from zip file
cliwoc_data = extract_logbook_data('CLIWOC15.csv')
#cliwoc_data = cliwoc_data.loc[50000:100000]

C:\Users\Emma\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3006: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
label_encoding = preprocessing.LabelEncoder().fit(cliwoc_data['LogbookIdent']).classes_
cliwoc_data['LogbookIdent'] = preprocessing.LabelEncoder().fit_transform(cliwoc_data['LogbookIdent'])

### Find definite slave data in CLIWOC data set
- These logs will be used to test the classifier

In [4]:
# extract logs that mention slaves
slave_mask = wc.count_key_words(cliwoc_data, text_columns, slave_words)
print('Found ', len(slave_mask[slave_mask==True]), ' logs that mention slaves')

Found  464  logs that mention slaves


### Clean CLIWOC data

In [5]:
# find indices of ship names that are "non-slave" ships before dropping ship name column
non_slave_log_locations = isolate_training_data(cliwoc_data, {'ShipName': non_slave_ships})
print('Found ', len(non_slave_log_locations[non_slave_log_locations==True]), ' logs that are non-slave ships')

Found  6517  logs that are non-slave ships


In [7]:
cliwoc_data['slave_logs'] = np.zeros(len(cliwoc_data))
slave_log_locations = cliwoc_data['LogbookIdent'].isin(list(cliwoc_data['LogbookIdent']
                                                            [slave_mask].unique()))

- cliwoc_data (unclassified) = 0
- cliwoc_data (no slaves)    = 1
- cliwoc_data (slaves)       = 2
- slave_data                 = 3

In [8]:
cliwoc_data.loc[non_slave_log_locations,'slave_logs'] = 1
cliwoc_data.loc[slave_log_locations,'slave_logs'] = 2

In [9]:
cliwoc_data = cliwoc_data.sort_values('LogbookIdent', ascending=True)
cliwoc_data = cliwoc_data.drop_duplicates('LogbookIdent')
cliwoc_data = cliwoc_data.set_index('LogbookIdent')

In [96]:
# uncomment this if looking at ship names for manual review
#desired_columns.append('ShipName')

['VoyageFrom',
 'VoyageTo',
 'ShipType',
 'Nationality',
 'Year',
 'slave_logs',
 'ShipName',
 'ShipName',
 'ShipName']

In [10]:
# remove undesired columns
cliwoc_data = isolate_columns(cliwoc_data, desired_columns)

### Load Slave Voyages data

In [11]:
file_name = './exploringShipLogbooks/data/tastdb-exp-2010'
slave_voyage_logs = pd.read_pickle(file_name)

In [12]:
year_ind = ~(slave_voyage_logs['yeardep'].isnull())
slave_voyage_logs = slave_voyage_logs[year_ind]

In [13]:
cliwoc_ind = (slave_voyage_logs['yeardep']>cliwoc_data['Year'].min()) & (slave_voyage_logs['yeardep']<cliwoc_data['Year'].max())
slave_voyage_logs = slave_voyage_logs[cliwoc_ind]

### Clean Slave voyages data

In [14]:
slave_voyage_desired_cols = list(slave_voyage_conversions.keys())
slave_voyage_logs = isolate_columns(slave_voyage_logs, slave_voyage_desired_cols)

slave_voyage_logs.rename(columns=slave_voyage_conversions, inplace=True)
#slave_voyage_logs.columns = ['Nationality', 'ShipType', 'VoyageFrom', 'VoyageTo', 'Year']

In [15]:
slave_voyage_logs['slave_logs'] = 3
slave_voyage_indices = range(len(slave_voyage_logs)) + (cliwoc_data.tail(1).index[0]+1)
slave_voyage_logs = slave_voyage_logs.set_index(slave_voyage_indices)

# Join data sets

In [16]:
all_data = pd.concat([cliwoc_data, slave_voyage_logs])
#all_data = cliwoc_data.append(slave_voyage_logs)
all_data = clean_data(all_data)

# cleanup (my computer won't run the code without this :( )
del cliwoc_data, slave_voyage_logs

## Test of fuzzywuzzy method

In [21]:
 df = pd.DataFrame( {'id':[1, 2, 3, 4, 5, 6], 'name':['dog', 'cat', 'mad cat', 'good dog', 'bad dog', 'chicken']})

In [22]:
def func(name):
    matches = df.apply(lambda row: (fuzz.partial_ratio(row['name'], name) >= 85), axis=1)
    return [i for i, x in enumerate(matches) if x]

In [23]:
df.apply(lambda row: func(row['name']), axis=1)

0    [0, 3, 4]
1       [1, 2]
2       [1, 2]
3       [0, 3]
4       [0, 4]
5          [5]
dtype: object

## Try fuzzywuzzy on subset of one of our columns

In [24]:
def func(name, column_name):
    matches = all_data[0:10].apply(lambda row: (fuzz.partial_ratio(row[column_name], name) >= 85), axis=1)
    return [i for i, x in enumerate(matches) if x]

In [ ]:
column_name = 'Nationality'
all_data[0:10].apply(lambda row: func(row[column_name], column_name), axis=1)

# Encode data
- Must encode data before separating, otherwise values that do not occur in a subset will be encoded differently

In [17]:
all_data = encode_data_df(all_data)

## Remove nan columns from encoding (currently there are no empty cells)

In [18]:
all_data['no_data'] = all_data['nan'].apply(lambda x: x.any(), axis=1).astype(int)

In [19]:
all_data = all_data.drop('nan', axis=1)

In [29]:
all_data.head()

,american,argentina,brazil,british,danish,denmark,dutch,france,french,great britain,...,ysla de santa cathalina,ysla de santa cathalinaysla de santa cathalina,zeeland,ziam,zierikzee,zuid afrika,zuiden,Year,slave_logs,no_data
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1785,0,1
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1786,0,1
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1786,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1786,0,1
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1786,0,1


# Extract training data, and create list of classes

In [20]:
unclassified_logs = all_data[all_data['slave_logs']==0]
unclassified_logs = unclassified_logs.drop('slave_logs', axis=1)

validation_set_1 = all_data[all_data['slave_logs']==2]
validation_set_1 = validation_set_1.drop('slave_logs', axis=1)

# reserve first 20% of slave_voyage_logs as validation set
validation_set_2_indices = range(slave_voyage_indices.min(),
                                 slave_voyage_indices.min() + round(len(slave_voyage_indices)*.2))
validation_set_2 = all_data.iloc[validation_set_2_indices]
validation_set_2 = validation_set_2.drop('slave_logs', axis=1)

training_logs_pos = all_data.drop(validation_set_2_indices)
training_logs_pos = training_logs_pos[training_logs_pos['slave_logs']==3]
training_logs_pos = training_logs_pos.drop('slave_logs', axis=1)

# note! This relies on cliwoc data being first in all_data
# could make more robust later
training_logs_neg = all_data[all_data['slave_logs']==1]
training_logs_neg = training_logs_neg.drop('slave_logs', axis=1)

# cleanup
#del all_data

In [22]:
len(training_logs_pos)/len(training_logs_neg)

727.8888888888889

- left this code so we can check if there are any null values in each 
  dataframe

In [ ]:
def finding_null_values(df):
    return df.isnull().sum()[df.isnull().sum()>0]

In [29]:
repeat_multiplier = round(len(training_logs_pos)/len(training_logs_neg))

# create list of classes for training data (0 is for non-slave, 1 is for slave)
# index matches training_data
classes = np.zeros(len(training_logs_neg)).repeat(repeat_multiplier)
#classes = np.append(classes, np.ones(len(training_logs_pos)))
classes = np.append(classes, np.ones(len(training_logs_pos)))

# join training data
neg_rep = pd.concat([training_logs_neg]*repeat_multiplier)
training_data = pd.concat([neg_rep, training_logs_pos], ignore_index = True)

# convert to numpy array
training_data = training_data.as_matrix()

# Fit training data to classifier
- **note!** first column of numpy array is index! do not include in classification!

In [24]:
classifier = MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
classifier.fit(training_data[::,1::], classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Test classifier
- check if slave logs from cliwoc data were classified correctly (want mostly classified as 1)
- compare first column with slave_index


In [25]:
def validation_test(classifier, validation_set, expected_class):
    """
    input classifer object, validation set (data frame), and expected class 
    of validation set (i.e. 1 or 0). Prints successful classification rate.
    """
    validation_set = validation_set.as_matrix()
    predictions = classifier.predict(validation_set[::,1::])
    
    counts = collections.Counter(predictions)
    percent_correct = (counts[expected_class]/(len(predictions))* 100)
                       
    print('Validation set was classified as', expected_class,
          round(percent_correct,2), '% of the time')

In [26]:
def predict_class(classifier, data_subset):
    """
    Predict class of data, and append predictions to data frame
    """
    try:
        # drop old predictions before reclassifying (if they exist)
        data_subset = data_subset.drop('predictions', axis = 1)
        data_to_classify = data_subset.copy()
    except:
        data_to_classify = data_subset.copy()
        pass
    
    # convert to numpy and classify
    data_matrix = data_to_classify.as_matrix()
    predictions = classifier.predict(data_matrix[::,1::])
    
    # append predictions to dataframe
    data_to_classify['predictions'] = predictions
    
    # print statstics
    counts = collections.Counter(predictions)
    
    for key in counts:
        percent = (counts[key]/(len(predictions))* 100)
        print(round(percent, 2), 'of data was classified as ', key)

    return data_to_classify
    
                       


In [27]:
print('Testing validation data from slave logs data set')
validation_test(classifier, validation_set_2, 1)

print('Testing validation data from cliwoc data set:')
validation_test(classifier, validation_set_1, 1)

Testing validation data from slave logs data set
Validation set was classified as 1 99.95 % of the time
Testing validation data from cliwoc data set:
Validation set was classified as 1 0.0 % of the time


In [28]:
unclassified_logs = append_predictions(classifier, unclassified_logs)

99.74 of data was classified as  0.0
0.26 of data was classified as  1.0
